In [ ]:
%pip install requests

In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on

In [2]:
import doctest

In [3]:
import os
import pathlib
import requests
import shutil
import tempfile
import time

SERVER_URL = 'http://localhost:8001'

URL_FMT = SERVER_URL + '/flags/{cc}/{cc}.gif'

DEST_DIR = 'downloads/'
if os.path.exists(DEST_DIR):
    shutil.rmtree(DEST_DIR)
    os.mkdir(DEST_DIR)
else:
    os.mkdir(DEST_DIR)


ALL_CC = (
    'AD AM AZ BF BN BW CF CM CV DK EG FJ GD GQ HN IE IS KE KN LA LR LY '
    'MG MN MW NE NP PE PT RS SC SK SR SZ TL TT UG VC YE AE AO BA BG BO '
    'BY CG CN CY DM ER FM GE GR HR IL IT KG KP LB LS MA MH MR MX NG NR '
    'PG PW RU SD SL SS TD TM TV US VE ZA AF AR BB BH BR BZ CH CO CZ DZ '
    'ES FR GH GT HT IN JM KH KR LC LT MC MK MT MY NI NZ PH PY RW SE SM '
    'ST TG TN TW UY VN ZM AG AT BD BI BS CA CI CR DE EC ET GA GM GW HU '
    'IQ JO KI KW LI LU MD ML MU MZ NL OM PK QA SA SG SN SV TH TO TZ UZ '
    'VU ZW AL AU BE BJ BT CD CL CU DJ EE FI GB GN GY ID IR JP KM KZ LK '
    'LV ME MM MV NA NO PA PL RO SB SI SO SY TJ TR UA VA WS'.split()
)

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR'.split())


def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):
    url = URL_FMT.format(cc=cc.lower())
    resp = requests.get(url)
    return resp.content


def show(text):
    print(text, end=' ')
    # sys.stdout.flush()


def main(download_many):
    t0 = time.perf_counter()
    count = download_many(ALL_CC)
    elapsed = time.perf_counter() - t0
    print()
    print(f'donwloaded {count} flags in {elapsed:.4f}s')

In [4]:
def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, f'{cc.lower()}.gif')

    return len(cc_list)


main(download_many)

AD AE AF AG AL AM AO AR AT AU AZ BA BB BD BE BF BG BH BI BJ BN BO BR BS BT BW BY BZ CA CD CF CG CH CI CL CM CN CO CR CU CV CY CZ DE DJ DK DM DZ EC EE EG ER ES ET FI FJ FM FR GA GB GD GE GH GM GN GQ GR GT GW GY HN HR HT HU ID IE IL IN IQ IR IS IT JM JO JP KE KG KH KI KM KN KP KR KW KZ LA LB LC LI LK LR LS LT LU LV LY MA MC MD ME MG MH MK ML MM MN MR MT MU MV MW MX MY MZ NA NE NG NI NL NO NP NR NZ OM PA PE PG PH PK PL PT PW PY QA RO RS RU RW SA SB SC SD SE SG SI SK SL SM SN SO SR SS ST SV SY SZ TD TG TH TJ TL TM TN TO TR TT TV TW TZ UA UG US UY UZ VA VC VE VN VU WS YE ZA ZM ZW 
donwloaded 194 flags in 1.6000s


In [5]:
from concurrent import futures
from timeit import timeit

MAX_WORKERS = 20


def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    num_workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
        # results = executor.map(download_one, sorted(cc_list))
        todo = []
        for cc in sorted(cc_list):
            future = executor.submit(download_one, cc)
            todo.append(future)
            print(f'scheduled for {cc}: {future}')

        results = []
        for future in futures.as_completed(todo):
            res = future.result()
            print(f'{future} result: {res!r}')
            results.append(res)

    return len(list(results))


main(download_many)

scheduled for AD: <Future at 0x7fcc580b7460 state=running>
scheduled for AE: <Future at 0x7fcc49c7eaf0 state=running>
scheduled for AF: <Future at 0x7fcc49ed8a90 state=running>
scheduled for AG: <Future at 0x7fcc49ee7ee0 state=running>
AD AF scheduled for AL: <Future at 0x7fcc49eeb040 state=running>
scheduled for AM: <Future at 0x7fcc49cb84f0 state=running>
AE AG scheduled for AO: <Future at 0x7fcc49ed8040 state=running>
scheduled for AR: <Future at 0x7fcc49eeb190 state=pending>
scheduled for AT: <Future at 0x7fcc49c533d0 state=pending>AL 
AO scheduled for AU: <Future at 0x7fcc49c7ec10 state=pending>AM 
scheduled for AZ: <Future at 0x7fcc49c7ebe0 state=running>ATAR  
scheduled for BA: <Future at 0x7fcc580f34f0 state=pending>
scheduled for BB: <Future at 0x7fcc49cb8610 state=pending>
AZ scheduled for BD: <Future at 0x7fcc49cbaf40 state=running>
scheduled for BE: <Future at 0x7fcc49c5d9a0 state=pending>
scheduled for BF: <Future at 0x7fcc49c5d7c0 state=pending>
AUscheduled for BG: <Futur

In [6]:
from concurrent import futures
from timeit import timeit

MAX_WORKERS = 20


def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc


def download_many(cc_list):
    with futures.ProcessPoolExecutor() as executor:
        results = executor.map(download_one, sorted(cc_list))

    return len(list(results))


main(download_many)

AD AEAF AG   ALAM AR AT  AOAUAZ  BB BA BDBF BE BG  BH BI BJ BN BO BRBS  BT BW BY BZ CA CD CF CG CH CI CLCM  CN CO CRCU  CV CY CZ DE DJDK  DM DZ EC EE EG ER ESET  FI FJ FM FR GA GB GD GE GH GM GN GQ GR GT GW GY HN HR HT HU ID IE IL IN IR IQ IS ITJM  JO JP KE KGKH  KI KN KM KR KP KW KZ LA LB LC LI LK LR LS LT LU LV LY MA MC MD ME MG MH MK ML MM MN MR MT MU MV MW MX MY MZ NA NENG  NINL  NO NP NR NZ OM PA PG PE PH PK PL PT PW PY QA RO RS RU RW SA SB SC SD SE SG SI SK SL SM SN SO SR SS ST SV SY SZ TDTG  TH TJ TL TM TN TO TT TR TV TW TZ UA UG US UZ UY VA VC VE VN VU WS YE ZA ZM ZW 
donwloaded 194 flags in 1.5326s


In [7]:
from time import sleep, strftime
from concurrent import futures


def display(*args):
    print(strftime('[%H:%M:%S]'), end=' ')
    print(*args)


def loiter(n):
    msg = '{}loiter({}): doing nothing for {}s...'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}): done'
    display(msg.format('\t'*n, n))
    return n * 10


display('script starting')
executor = futures.ThreadPoolExecutor(max_workers=3)
results = executor.map(loiter, range(5))
display('results:', results)
display('waiting for individual results:')
for i, result in enumerate(results):
    display('result {}: {}'.format(i, result))

[05:29:07] script starting
[05:29:07] loiter(0): doing nothing for 0s...
[05:29:07] loiter(0): done
[05:29:07] 	loiter(1): doing nothing for 1s...
[05:29:07][05:29:07] 			loiter(3): doing nothing for 3s...
 		loiter(2): doing nothing for 2s...
[05:29:07] results: <generator object Executor.map.<locals>.result_iterator at 0x7fcc49cb5ac0>
[05:29:07] waiting for individual results:
[05:29:07] result 0: 0
[05:29:08] 	loiter(1): done
[05:29:08] 				loiter(4): doing nothing for 4s...
[05:29:08] result 1: 10
[05:29:09] 		loiter(2): done
[05:29:09] result 2: 20
[05:29:10] 			loiter(3): done
[05:29:10] result 3: 30
[05:29:12] 				loiter(4): done
[05:29:12] result 4: 40
